In [1]:
import datetime as dt

import numpy as np
import pandas as pd
import pytz

Useful timezone database (`pandas` depends on this package)

In [2]:
len(pytz.common_timezones)

440

In [3]:
[x for x in pytz.common_timezones if 'US' in x]

['US/Alaska',
 'US/Arizona',
 'US/Central',
 'US/Eastern',
 'US/Hawaii',
 'US/Mountain',
 'US/Pacific']

In [4]:
len(pytz.all_timezones)

593

Also `dateutil` but depends on OS timezones so less recommended

In [5]:
# datetimes are, by default, naive (no specified timezone)
idx = pd.date_range('2001-01-01', periods=2, freq='H')
idx

DatetimeIndex(['2001-01-01 00:00:00', '2001-01-01 01:00:00'], dtype='datetime64[ns]', freq='H')

In [6]:
# same as naive datetimes assumed to be UTC
idx_utc = idx.tz_localize('UTC')
idx_utc

DatetimeIndex(['2001-01-01 00:00:00+00:00', '2001-01-01 01:00:00+00:00'], dtype='datetime64[ns, UTC]', freq='H')

In [7]:
# always store as UTC and convert to local times
idx_utc.tz_convert('US/Eastern')

DatetimeIndex(['2000-12-31 19:00:00-05:00', '2000-12-31 20:00:00-05:00'], dtype='datetime64[ns, US/Eastern]', freq='H')

In [8]:
# convert to `datetime` objects
idx_utc.timetz

array([datetime.time(0, 0, tzinfo=<UTC>),
       datetime.time(1, 0, tzinfo=<UTC>)], dtype=object)

In [9]:
idx_utc.tz

<UTC>

In [10]:
# example of US DST
dst_ts = pd.Timestamp('2019-11-03 01:00')

In [11]:
try:
    dst_ts.tz_localize('US/Eastern')
except pytz.AmbiguousTimeError as e:
    print(e)

Cannot infer dst time from 2019-11-03 01:00:00, try using the 'ambiguous' argument


In [12]:
dst_ts.tz_localize('US/Eastern', ambiguous='NaT')

NaT

In [13]:
# pandas can infer time when you have duplicates
pd.Index([
    pd.Timestamp('2019-11-03 00:00'),
    pd.Timestamp('2019-11-03 01:00'),
    pd.Timestamp('2019-11-03 01:00')
]).tz_localize('US/Eastern', ambiguous='infer')

DatetimeIndex(['2019-11-03 00:00:00-04:00', '2019-11-03 01:00:00-04:00',
               '2019-11-03 01:00:00-05:00'],
              dtype='datetime64[ns, US/Eastern]', freq=None)

In [14]:
# also handle cases where the time is not real from the Pandas documentation
# https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#nonexistent-times-when-localizing
dti = pd.date_range(start='2015-03-29 02:30:00', periods=3, freq='H')

In [15]:
dti

DatetimeIndex(['2015-03-29 02:30:00', '2015-03-29 03:30:00',
               '2015-03-29 04:30:00'],
              dtype='datetime64[ns]', freq='H')

In [16]:
try:
    dti.tz_localize('Europe/Warsaw')
except pytz.NonExistentTimeError as e:
    print(f'NonExistentTimeError: {e}')

NonExistentTimeError: 2015-03-29 02:30:00


In [17]:
dti.tz_localize('Europe/Warsaw', nonexistent='shift_backward')

DatetimeIndex(['2015-03-29 01:59:59.999999999+01:00',
                         '2015-03-29 03:30:00+02:00',
                         '2015-03-29 04:30:00+02:00'],
              dtype='datetime64[ns, Europe/Warsaw]', freq='H')